In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
'''
prepare a text message in JSON & transfer into TextSendMessage & send to users
'''
jsonString = '''
{
  "type": "text",
  "text": "お前のドリルでテッペンをつけ！"
}
'''

import json
jsonDict = json.loads(jsonString)

print(jsonDict)

{'type': 'text', 'text': 'お前のドリルでテッペンをつけ！'}


In [3]:
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# place jsonDict into the func
jsonDictToTextSendMessage = TextSendMessage.new_from_json_dict(jsonDict)

In [4]:
# let handler to do below func when received text msg from a user
@handler.add(MessageEvent, message=TextMessage)
def replyText(event):
    line_bot_api.reply_message(
    event.reply_token,
    jsonDictToTextSendMessage  
    )

In [5]:
app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [16/Sep/2019 02:51:06] "POST / HTTP/1.1" 200 -
